# Information
## Source
This notebook prepares the data from Lawley et al. (2022) using the version from https://drive.google.com/file/d/1jyxbPmwhMEhgezxMTxwmKTuU1PhT9yPe using the original H3 hexagonal polygon data.

**Citaton**: <p>
Christopher J.M. Lawley, Anne E. McCafferty, Garth E. Graham, David L. Huston, Karen D. Kelley, Karol Czarnota, Suzanne Paradis, Jan M. Peter, Nathan Hayward, Mike Barlow, Poul Emsbo, Joshua Coyan, Carma A. San Juan, Michael G. Gadd: <br>
Data–driven prospectivity modelling of sediment–hosted Zn–Pb mineral systems and their critical raw materials. <br>
Ore Geology Reviews, Volume 141, 2022, 104635, ISSN 0169-1368, https://doi.org/10.1016/j.oregeorev.2021.104635.

## What is it for?
This notebook contains the functionality to export the datacube data into a raster format for given columns.<br>
The extracted raster will serve as initial input to test the SOM-approach.

The functions also return a **tuple** for extracted **columns**, **Numpy arrays** and **raster transformation** information.

## How to
1. Get the datacube from the link above and extract the .csv file. In this notebook, we use *data/LAWLEY22-RAW*.<p>
2. Read the *Model setup* section<p>
3. Choose the preferred model from the Lawley et al. (2022) paper. Since we're working with the MVT, the definition is based on the **preferred** MVT model from the paper. <br>Your'e welcome to add other models in the <code>model.py</code> as you want to. The notebook will export all **keys** with a **True** value, but not those with a **False** one. This way, it's easy to change exports as desired.<p>
4. Make some **user-defined** decisions and adjust path names etc. if necessary. <br>The source CRS is based on the EPSG:4326 for the given dataset and does not need any change. <p>**ALL** possible user-interactions are in uppercase-letters.<br>The only point not considered here are the **Training** columns which may need some change if you want to export data for a **CD** model. You can do so down below in the **ground truth** investigation cells. Simply change *Training_MVT* to *Training_CD* if necessary.<p>
Default resolution based on the EPSG:4326 CRS is 0.05 degree. <br>
**We have to mention, that the generation of proxies like distance calculations, first and second order derivatives etc. will not work correctly with a geopgraphic CRS**.
5. Check the content of the datacube within the **Data** section.<p>
6. Export the data of the datacube for
    - numerical
    - categorical
    - ground truth data<p>

    Categorical columns will be exported based on their unique values into binary rasters. I.e., if the column *Geology_Period_Maximum_Majority* contains <code>x</code> distinct values like *Cretaceous, Paleogene, Jurassic and Devonian*, the export of this column will result in 4 raster files with **1** where the specific value is given (e.g. where <code>Geology_Period_Maximum_Majority == Jurassic</code>) and **0** where not (e.g. where <code>Geology_Period_Maximum_Majority != Jurassic</code>).<p>
    For **Present**/**Absent** coded columns, only the **Present** information will be exported, since the binary encoding implies that **Absend** is represented by **0**.<p>
    For export, the three function calls for the <code>rasterize_vectors</code> respective data types may need some **user-adjustments** depending on the desired result (e.g. <code>dtype</code>, <code>nodata_value</code> etc.).<br>
    For numerical columns, we usually use <code>float32</code>. For categorical and ground truth data, <code>uint8</code> with a **negative** nodata value


## Model setup

Note that the **Geology_Dictionary** categories contain more than the incorporated dictionaries. Lawley et al. (2022) explicitly stated that:
- Sedimentary dictionaries = FineClastic, Carbonaceous, Calcareous
- Igneous dictionaries = Felsic, Intermediate, Mafic, Ultramafic
- Metamorphic dictionaries = Anatectic, Gneissose, Schistose

However, the complete list would be like:
- Sedimentary dictionaries = FineClastic, Carbonaceous, Calcareous, **Cherty, CoarseClastic, Evaporitic, RedBed, Sedimentary**
- Igneous dictionaries = Felsic, Intermediate, Mafic, Ultramafic, **Alkalic, Pegmatitic**
- Metamorphic dictionaries = Anatectic, Gneissose, Schistose

Also note, that the **Mafic** dictionary **does not exist** as separate column in the datacube and thus was not considered in the model setup!

In [1]:
# Standard libraries
import numpy as np
import pandas as pd
from pathlib import Path

# Custom modules
import utilities as utils # Custom functions for processing the datacube
from models import models # Includes the model definitions used in this notebook 


# Definitions
**User inputs**

In [2]:
# Choose model
MODEL = "MVT_PREFERRED"

# Path to datacube and export folder
PATH_DATACUBE_IN = "data/LAWLEY22-RAW/2021_Table04_Datacube.csv"
PATH_RASTER_OUT = "data/LAWLEY22-DATACUBE-EXPORT/" + MODEL
PATH_TEMP_OUT = "data/TEMP/"

# Folder names
NAME_NUMERICAL = "NUMERICAL"
NAME_CATEGORICAL = "CATEGORICAL"
NAME_GROUND_TRUTH = "GROUND_TRUTH"

# ROI
REGIONS =["United States of America", "Canada"]     # Canada, United States of America, Australia
N_ROWS = None                                       # Number of rows to read from datacube, None for all

# Coordinate system
EPSG_SRC = 4326
EPSG_PRJ = 3857

# Resolution according to the CRS
RES_SRC = 0.05
RES_PRJ = 5000

# COLUMN NAMES
COL_LONGITUDE = "Longitude_EPSG4326"
COL_LATITUDE = "Latitude_EPSG4326"
COL_POLYOGNS = "H3_Geometry"


Definitions

In [4]:
# Path variables
datacube = Path(PATH_DATACUBE_IN)
export_folder = utils.check_path(Path(PATH_RASTER_OUT))

path_numerical_data = Path(PATH_RASTER_OUT) / NAME_NUMERICAL
numerical_data = utils.check_path(path_numerical_data)

path_categorical_data = Path(PATH_RASTER_OUT) / NAME_CATEGORICAL
categorical_data = utils.check_path(path_categorical_data)

path_ground_truth_data = Path(PATH_RASTER_OUT) / NAME_GROUND_TRUTH
ground_truth_data = utils.check_path(path_ground_truth_data)

# Coordinate columns
long_col = COL_LONGITUDE
lat_col = COL_LATITUDE

# Parameters for rasterization
target_epsg = EPSG_SRC
resolution = EPSG_PRJ

# Relevant columns for choosen model (considering only True values)
model = models[MODEL]
model_columns = [key for key, value in model.items() if value == True]


Load RAW datacube with selection to U.S. and Canada

In [5]:
# Load datacube 
df = utils.load_dataset(datacube, nrows=N_ROWS)
df = df[df["Country_Majority"].isin(REGIONS)]
df.replace("-", np.nan, inplace=True)

print(df.shape)


(3620129, 97)


Some environment settings

In [6]:
# Allow only 3 decimals in DataFrame view
pd.options.display.float_format = "{:.3f}".format


# Data

Create some general datacube insights

In [7]:
# Hash geometry columns 
geometry_cols = df.columns[df.columns.str.contains("H3|Latitude|Longitude")].tolist()
geometry_cols


['ï»¿"H3_Address"',
 'H3_Resolution',
 'H3_Geometry',
 'Longitude_EPSG4326',
 'Latitude_EPSG4326']

Investigate numerical columns

In [8]:
# Hash numerical columns
numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
numerical_cols = [col for col in numerical_cols if col not in geometry_cols]

df_numerical_features = pd.DataFrame({"Column Name": numerical_cols, "Source-Index": [df.columns.get_loc(col) for col in numerical_cols]})

# Append summary statistics to numerical_cols_df
percentiles = [0.05, 0.25, 0.5, 0.75, 0.95]
numerical_summary = df[numerical_cols].describe(percentiles=percentiles).T[["mean", "std", "min", "5%", "25%", "50%", "75%", "95%", "max"]]
df_numerical_features = pd.merge(df_numerical_features, numerical_summary, left_on="Column Name", right_index=True)

# Append missing values cound
missing_counts = df[numerical_cols].isnull().sum()
df_numerical_features["NaN"] = missing_counts.values

# Naming
df_numerical_features = df_numerical_features.rename(columns=lambda x: x.capitalize())

# Output
df_numerical_features

,Column name,Source-index,Mean,Std,Min,5%,25%,50%,75%,95%,Max,Nan
0,Terrane_Proximity,14,58.108,55.320,0.001,3.082,17.650,42.133,81.188,168.200,883.486,0
1,Geology_PassiveMargin_Proximity,48,313.184,201.321,0.420,31.053,150.109,295.047,455.324,656.270,4295.096,0
2,Geology_BlackShale_Proximity,49,191.727,288.059,0.000,7.005,28.477,75.125,210.421,885.245,2308.477,0
3,Geology_Fault_Proximity,50,68.636,90.487,0.000,1.267,8.574,28.994,90.520,285.292,511.958,0
4,Geology_CoverThickness,51,17.169,19.266,0.000,0.266,1.000,5.689,35.903,50.000,50.000,0
5,Geology_Paleolongitude_Period_Maximum,52,-70.879,40.503,-175.870,-154.990,-90.260,-63.180,-37.520,-22.640,173.480,1112814
6,Geology_Paleolongitude_Period_Minimum,53,-75.601,39.941,-175.830,-154.720,-97.220,-71.430,-48.720,-17.460,176.150,884615
7,Geology_Paleolatitude_Period_Maximum,54,24.279,31.577,-81.970,-26.170,-7.910,37.140,47.250,61.120,82.670,1112814
8,Geology_Paleolatitude_Period_Minimum,55,27.479,32.871,-81.970,-36.610,-2.960,40.970,56.010,66.050,82.670,884615
9,Seismic_LAB_Hoggard,56,149.603,54.187,35.215,46.292,124.803,169.630,188.723,211.277,245.492,0


Investigate categorical columns

In [9]:
# Hash categorical columns
categorical_cols = df.select_dtypes(include=object).columns.tolist()
categorical_cols = [col for col in categorical_cols if col not in geometry_cols and "Training" not in col]

df_categorical_features = pd.DataFrame({"Column Name": categorical_cols, "Source-Index": [df.columns.get_loc(col) for col in categorical_cols]})

# Append summary statistics to categorical_cols_df
categorical_summary = df[categorical_cols].describe().T[["unique"]]
categorical_summary = categorical_summary.rename(columns={"unique": "Classes"})

# Store unique values as a dictionary
unique_values_dict = {}
for column in categorical_cols:
  unique_values_dict[column] = df[column].unique().tolist()

# Append unique values to categorical_summary dataframe
categorical_summary["Values"] = categorical_summary.index.map(unique_values_dict)

# Merge dataframes
df_categorical_features = pd.merge(df_categorical_features, categorical_summary, left_on="Column Name", right_index=True)

# Naming
df_categorical_features = df_categorical_features.rename(columns=lambda x: x.capitalize())

# Append missing values count
missing_counts = df[categorical_cols].isnull().sum()
df_categorical_features["NaN"] = missing_counts.values

# Output
df_categorical_features


,Column name,Source-index,Classes,Values,NaN
0,Continent_Majority,5,1,"[North America, nan]",107
1,Continent_Minority,6,1,"[North America, nan]",107
2,Country_Majority,7,2,"[Canada, United States of America]",0
3,Country_Minority,8,3,"[Canada, United States of America, Mexico]",0
4,Province_Majority,9,63,"[Alberta, British Columbia, Northwest Territor...",0
5,Province_Minority,10,70,"[Alberta, British Columbia, Saskatchewan, Nort...",0
6,Terrane_Majority,11,211,"[Buffalo Head, Wabamun, Chinchaga, Hearne (cov...",0
7,Terrane_Minority,12,211,"[Buffalo Head, Wabamun, Chinchaga, Hearne (cov...",0
8,Terrane_Contact,13,2,"[Absent, Present]",0
9,Geology_Eon_Maximum_Majority,15,2,"[Phanerozoic, Precambrian, nan]",44289


Investigate ground truth data

In [11]:
# Create combined (max) column for training points 
df["Training_MVT"] = df.apply(lambda row: "Present" if "Present" in [row["Training_MVT_Deposit"], row["Training_MVT_Occurrence"]] else "Absent", axis=1)


In [12]:
# Hash ground truth columns
ground_truth_cols = [col for col in df.columns if "Training_MVT" in col]
df_ground_truth_features = pd.DataFrame({"Column Name": ground_truth_cols, "Source-Index": [df.columns.get_loc(col) for col in ground_truth_cols]})

# Append summary statistics to ground_truth_cols_df
ground_truth_summary = df[ground_truth_cols].describe().T[["unique"]]
ground_truth_summary = ground_truth_summary.rename(columns=lambda x: x.capitalize())

# Store unique values as a dictionary
unique_values_dict = {}
for column in ground_truth_cols:
  unique_values_dict[column] = df[column].unique().tolist()
  
# Append unique values to ground_truth_summary dataframe
ground_truth_summary["Values"] = ground_truth_summary.index.map(unique_values_dict)

# Merge dataframes
df_ground_truth_features = pd.merge(df_ground_truth_features, ground_truth_summary, left_on="Column Name", right_index=True)
df_ground_truth_features = df_ground_truth_features.rename(columns=lambda x: x.capitalize())

# Append missing values count
missing_counts = df[ground_truth_cols].isnull().sum()
df_ground_truth_features["NaN"] = missing_counts.values

# Output
df_ground_truth_features


,Column name,Source-index,Unique,Values,NaN
0,Training_MVT_Deposit,93,2,"[Absent, Present]",0
1,Training_MVT_Occurrence,94,2,"[Absent, Present]",0
2,Training_MVT,97,2,"[Absent, Present]",0


# Export to raster format

## Convert H3 polygons and create geodataframe

In [13]:
# Create geodataframe in advance
gdf = utils.create_geodataframe_from_polygons(data=df, polygon_col=COL_POLYOGNS, epsg_code=EPSG_SRC)

## Export **numerical** columns

In [14]:
# Select relevant columns
model_columns_numerical = [col for col in model_columns if col in numerical_cols]

# Create and export rasters
_, _ , _ = utils.rasterize_vector(value_type="numerical",
                                  value_columns=model_columns_numerical,
                                  geodataframe=gdf,
                                  nodata_value=-99999,
                                  resolution=resolution,
                                  epsg_code=target_epsg,
                                  dtype=np.float32,
                                  raster_save=True,
                                  raster_save_folder=path_numerical_data)

Number of threads rasterizing: 48


Rasterizing: 100%|██████████| 17/17 [08:43<00:00, 30.78s/it]  


## Export **categorical** columns to binary rasters

The **class values** of each categorical column will be extracted and saved or returned as binary raster with 1 for the **actual class** at its location and 0 where **other classes** are present.<br>
Since some values contain characters not suitable for at least the Microsoft Windows world (e.g. "**/**" and other working but unpleasent like "**,**"), those will be replaced by "**_**" for saving.

In [15]:
# Select relevant columns
model_columns_categorical = [col for col in model_columns if col in categorical_cols]

# Create and export rasters
_, _, _ = utils.rasterize_vector(value_type="categorical",
                                 value_columns=model_columns_categorical,
                                 geodataframe=gdf,
                                 nodata_value=-99,
                                 resolution=resolution,
                                 epsg_code=target_epsg,
                                 dtype=np.int8,
                                 raster_save=True,
                                 raster_save_folder=Path(path_categorical_data) / column)


Number of threads rasterizing: 48


Rasterizing: 100%|██████████| 110/110 [33:39<00:00, 18.35s/it]   


## Export **ground truth** columns to binary rasters

Keeps only the **present** data as binary grids with 1 for **present** and 0 for **absent**.

In [16]:
_, _, _ = utils.rasterize_vector(value_type="ground_truth",
                                 value_columns=ground_truth_cols,
                                 geodataframe=gdf,
                                 nodata_value=-99,
                                 resolution=resolution,
                                 epsg_code=target_epsg,
                                 dtype=np.int8,
                                 raster_save=True,
                                 raster_save_folder=path_ground_truth_data)

Number of threads rasterizing: 48


Rasterizing: 100%|██████████| 3/3 [06:28<00:00, 129.46s/it]
